# DP Tensor Experiment - v7

### Purpose:
Vectorize v6 - will referneec v5

### Conclusions:
Seemed to work great.


In [1]:
import torch as th

In [13]:
max_long = 2**62

class PrivateTensor():
    
    def __init__(self, values, max_vals, min_vals, entities=None):
        
        self.values = values
        self.max_vals = max_vals
        self.min_vals = min_vals
        
        if(entities is None):
            entities = self.max_vals != self.min_vals
        
        # one hot encoding of entities in the ancestry of this tensor
        self.entities = entities
        
    def __add__(self, other):
        
        # add to a private number
        if(isinstance(other, PrivateTensor)):
            new_vals = self.values + other.values
            new_max_vals = (self.max_vals * self.entities) + (other.max_vals * other.entities)
            new_min_vals = (self.min_vals * self.entities) + (other.min_vals * other.entities)
            
        else:
            # add to a public number
            new_vals = self.values + other
            new_max_vals = self.max_vals + other
            new_min_vals = self.min_vals + other
        
        return PrivateTensor(new_vals,
                    new_max_vals,
                    new_min_vals) 
    
    def __sub__(self, other):
        
        # add to a private number
        if(isinstance(other, PrivateTensor)):
            
            new_vals = self.values - other.values
            
            # note that other.max/min values are reversed on purpose
            # because this functionality is equivalent to
            # output = self + (other * -1) and multiplication by
            # a negative number swaps the max/min values with each
            # other and flips their sign
            new_max_vals = (self.entities * self.max_vals) - (other.entities * other.min_vals)
            new_min_vals = (self.entities * self.min_vals) - (other.entities * other.max_vals)
            
        else:
            # add to a public number
            new_vals = self.values - other
            new_max_vals = self.max_vals - other
            new_min_vals = self.min_vals - other
        
        return PrivateTensor(new_vals,
                    new_max_vals,
                    new_min_vals) 
    
    def __mul__(self, other):
        
        if(isinstance(other, PrivateTensor)):
            
            new_vals = self.values * other.values
            
            new_self_max_vals = th.max(self.min_vals * other.expanded_minminvals,
                                         self.max_vals * other.expanded_maxmaxvals)
            
            new_self_min_vals = th.min(self.min_vals * other.expanded_maxmaxvals,
                                         self.max_vals * other.expanded_minminvals)
            
            
            new_other_max_vals = th.max(other.min_vals * self.expanded_minminvals,
                                          other.max_vals * self.expanded_maxmaxvals)
            
            new_other_min_vals = th.max(other.min_vals * self.expanded_maxmaxvals,
                                          other.max_vals * self.expanded_minminvals)
            
            
            entities_self_or_other = (self.entities + other.entities) > 0
            
            new_self_max_vals = (new_self_max_vals * self.entities) + ((1 - self.entities) * -max_long)
            new_other_max_vals = (new_other_max_vals * self.entities) + ((1 - other.entities) * -max_long)
            
            new_max_vals = th.max(new_self_max_vals, new_other_max_vals) * entities_self_or_other
            
            new_self_min_vals = (new_self_min_vals * self.entities) + ((1 - self.entities) * max_long)
            new_other_min_vals = (new_other_min_vals * self.entities) + ((1 - other.entities) * max_long)
            
            new_min_vals = th.min(new_self_min_vals, new_other_min_vals) * entities_self_or_other
            
        
        else:
            # add to a public number
            new_vals = self.values * other

            if(other > 0):
                new_max_vals = self.max_vals * other
                new_min_vals = self.min_vals * other
            else:
                new_min_vals = self.max_vals * other
                new_max_vals = self.min_vals * other
        
        return PrivateTensor(new_vals,
                    new_max_vals,
                    new_min_vals) 
    
    def __neg__(self):
        
        # note that new_min_vals and new_max_vals are reversed intentionally
        return PrivateTensor(-self.values,
                    -self.min_vals,
                    -self.max_vals) 

    def __truediv__(self, other):
        
        if(isinstance(other, PrivateTensor)):
            raise Exception("probably best not to do this - it's gonna be inf a lot")
            
        new_vals = self.values / other
        new_max_vals = self.max_vals / other
        new_min_vals = self.min_vals / other
        
        return PrivateTensor(new_vals,
                                new_max_vals,
                                new_min_vals)
    
    def __gt__(self, other):
        """BUG: the zero values mess this up"""
        if(isinstance(other, PrivateTensor)):
        
            new_vals = self.values > other.values
        
            # if self is bigger than the biggest possible other
            if_left = (self.min_vals > other.expanded_maxmaxvals).float() * self.entities
            
            # if self is smaller than the smallest possible other
            if_right = (self.max_vals < other.expanded_minminvals).float() * self.entities
            
            # if self doesn't overlap with other at all
            if_left_or_right = if_left + if_right # shouldn't have to check if this > 2 assuming
                                                  # other's max is > other's min
                
            # if self does overlap with other
            new_self_max_vals = 1 - if_left_or_right
            
            # can't have a threshold output less than 0
            new_self_min_vals = if_left_or_right * 0
            
            # if other is bigger than the smallest possible self
            if_left = (other.min_vals > self.expanded_maxmaxvals).float() * other.entities
            
            # if other is smaller than the smallest possible self
            if_right = (other.max_vals < self.expanded_minminvals).float() * other.entities
            
            # if other and self don't overlap
            if_left_or_right = if_left + if_right # shouldn't have to check if this > 2 assuming
                                                  # other's max is > other's min
        
            # if other and self do overlap
            new_other_max_vals = 1 - if_left_or_right
            
            # the smallest possible result is 0
            new_other_min_vals = new_self_min_vals + 0
        
            # only contribute information from entities in ancestry
            new_self_max_vals = (new_self_max_vals * self.entities) + ((1 - self.entities) * -max_long)
            new_other_max_vals = (new_other_max_vals * self.entities) + ((1 - self.entities) * -max_long)
        
            # only contribute information from entities in ancestry
            new_self_min_vals = (new_self_min_vals * self.entities) + ((1 - self.entities) * max_long)
            new_other_min_vals = (new_other_min_vals * self.entities) + ((1 - self.entities) * max_long)
            
            entities_self_or_other = ((self.entities + other.entities) > 0).float()
            
            new_max_val = th.max(new_self_max_vals, new_other_max_vals) * entities_self_or_other
            new_min_val = th.min(new_self_min_vals, new_other_min_vals) * entities_self_or_other

        else:

            new_vals = self.values > other
            
            if_left = other <= self.max_vals
            if_right = other >= self.min_vals
            if_and = if_left * if_right
            
            new_max_val = if_and
            new_min_val = new_max_val * 0
            
        return PrivateTensor(new_vals,
                             new_max_val,
                             new_min_val)
    
    def __lt__(self, other):
        
        if(isinstance(other, PrivateTensor)):
        
            result = self.values > other.values
        
            # if self is bigger than the biggest possible other
            if_left = (self.min_vals > other.expanded_maxmaxvals).float() * self.entities
            
            # if self is smaller than the smallest possible other
            if_right = (self.max_vals < other.expanded_minminvals).float() * self.entities
            
            # if self doesn't overlap with other at all
            if_left_or_right = if_left + if_right # shouldn't have to check if this > 2 assuming
                                                  # other's max is > other's min
                
            # if self does overlap with other
            new_self_max_vals = 1 - if_left_or_right
            
            # can't have a threshold output less than 0
            new_self_min_vals = if_left_or_right * 0
            
            
            # if other is bigger than the smallest possible self
            if_left = (other.min_vals > self.expanded_maxmaxvals).float() * other.entities
            
            # if other is smaller than the smallest possible self
            if_right = (other.max_vals < self.expanded_minminvals).float() * other.entities
            
            # if other and self don't overlap
            if_left_or_right = if_left + if_right # shouldn't have to check if this > 2 assuming
                                                  # other's max is > other's min
        
            # if other and self do overlap
            new_other_max_vals = 1 - if_left_or_right
            
            # the smallest possible result is 0
            new_other_min_vals = new_self_min_vals + 0
                
            # only contribute information from entities in ancestry
            new_self_max_vals = (new_self_max_vals * self.entities) + ((1 - self.entities) * -max_long)
            new_other_max_vals = (new_other_max_vals * self.entities) + ((1 - self.entities) * -max_long)
        
            # only contribute information from entities in ancestry
            new_self_min_vals = (new_self_min_vals * self.entities) + ((1 - self.entities) * max_long)
            new_other_min_vals = (new_other_min_vals * self.entities) + ((1 - self.entities) * max_long)
            
            entities_self_or_other = ((self.entities + other.entities) > 0).float()
            
            new_max_val = th.max(new_self_max_vals, new_other_max_vals) * entities_self_or_other
            new_min_val = th.min(new_self_min_vals, new_other_min_vals) * entities_self_or_other
        
        else:

            result = self.values < other
            
            if_left = other <= self.max_vals
            if_right = other >= self.min_vals
            if_and = if_left * if_right
            
            new_max_val = if_and
            new_min_val = new_max_val * 0
            
        return PrivateTensor(result,
                             new_max_val,
                             new_min_val)

    def clamp_min(self, other):
        
        if(isinstance(other, PrivateTensor)):
            raise Exception("Not implemented yet")
        
        new_min_val = self.min_vals.clamp_min(other)
            
        return PrivateTensor(self.values.clamp_min(other),
                                self.max_vals,
                                new_min_val)
    
    def clamp_max(self, other):
        
        if(isinstance(other, PrivateTensor)):
            raise Exception("Not implemented yet")
        
        entities = self.entities
        
        new_max_val = self.max_vals.clamp_max(other)
                
        return PrivateTensor(self.values.clamp_max(other),
                                new_max_val,
                                self.min_vals)
    
    @property
    def maxmaxvals(self):
        """This returns the maximum possible value over all entities"""
        
        return (self.max_vals * self.entities).sum(1)
    
    @property
    def expanded_maxmaxvals(self):
        return self.maxmaxvals.unsqueeze(1).expand(self.max_vals.shape)
    
    @property
    def minminvals(self):
        """This returns the minimum possible values over all entities"""

        return (self.min_vals * self.entities).min(1)[0]
    
    @property
    def expanded_minminvals(self):
        return self.minminvals.unsqueeze(1).expand(self.min_vals.shape)
    
    @property
    def sensitivity(self):
        return (self.max_vals - self.min_vals).sum(1)
    
    @property
    def entities(self):
        return self._entities
        
    @entities.setter
    def entities(self, x):
        self._entities = x.float()
        
    def hard_sigmoid(self):
        return self.min(1).max(0)
    
    def hard_sigmoid_deriv(self, leak=0.01):
        return ((self < 1) * (self > 0)) + (self < 0) * leak - (self > 1) * leak
    

In [14]:
data = th.tensor([2.,0,1,0,0,0])
max_vals = th.tensor([[3.,1,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,0,0], [0,0,0,0,1], [0,0,0,0,1]])
min_vals = th.tensor([[2.,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0], [0,0,0,0,0]])
x = PrivateTensor(values=data, 
                  max_vals=max_vals, 
                  min_vals=min_vals)

In [15]:
y = x.clamp_min(1)

In [16]:
y.values

tensor([2., 1., 1., 1., 1., 1.])

In [17]:
y.sensitivity

tensor([-2., -4., -4., -5., -4., -4.])

In [18]:
x.values.clamp_max(1)

tensor([1., 0., 1., 0., 0., 0.])

In [12]:
y.values

tensor([2., 1., 1., 1., 1., 1.])